In [1]:
import tensorflow as tf
import keras

input = tf.constant([[1,0,1],[0,1,0],[1,0,1]],shape=[1,3,3,1],dtype=tf.float32)
filter = tf.constant([[0.9,0.1],[0.1,0.9]],shape=[2,2,1,1],dtype=tf.float32)
b0 = tf.constant(0.1)
conv_output = tf.nn.conv2d(input,filter,strides=1,padding='VALID')+b0
w0 = filter

relu_output = tf.nn.relu(conv_output)
flat = tf.reshape(relu_output,[1,4])
x = flat
w1 = tf.constant([[0.1,0.5],[0.2,0.6],[0.3,0.7],[0.4,0.8]])
w2 = tf.constant([[0.1,0.3],[0.2,0.4]])
t = tf.constant([[1,0]])
b1 = tf.constant(0.2)
b2 = tf.constant(0.3)

def forward(x,w1,w2,b1,b2):
    sum1=tf.matmul(x,w1)+b1
    H = sum1
    sum2=tf.matmul(H,w2)+b2
    O = tf.nn.softmax(sum2)
    return O

def backward(x,w1,w2,b1,b2,t):
    with tf.GradientTape() as g:
       g.watch([w1,w2,x,b1,b2])
       O = forward(x, w1, w2, b1, b2)
       Etotal = -tf.math.log(O[:, 0])
       print("Total loss = " + str(Etotal))
    dEtotal_dw = g.gradient(Etotal, [w1,w2,x,b1,b2])
    dEtotal_dw1, dEtotal_dw2, dEtotal_dx, dEtotal_db1, dEtotal_db2 = dEtotal_dw
    return dEtotal_dw1,dEtotal_dw2,dEtotal_dx,dEtotal_db1,dEtotal_db2

dEtotal_dw1,dEtotal_dw2,dEtotal_dx,dEtotal_db1,dEtotal_db2 = backward(x,w1,w2,b1,b2,t)

w1 = w1 - 0.5*dEtotal_dw1 #the learning rate is set to be 0.5
w2 = w2 - 0.5*dEtotal_dw2

b1 = b1 - 0.5*dEtotal_db1
b2 = b2 - 0.5*dEtotal_db2

dfilter = tf.constant(dEtotal_dx.numpy(),shape=[2,2,1,1],dtype=tf.float32)
dEtotal_dw0 = tf.nn.conv2d(input,dfilter,strides = 1,padding ='VALID')
dEtotal_dw0 = tf.reshape(dEtotal_dw0,[2,2,1,1])
w0 = w0 - 0.5*dEtotal_dw0

dEtotal_db0 = tf.reduce_sum(dEtotal_dx)
b0 = b0 - 0.5*dEtotal_db0
print('The new w0 is',w0.numpy())
print('The new w1 is',w1.numpy())
print('The new w2 is',w2.numpy())
print('The new b0 is',b0.numpy())
print('The new b1 is',b1.numpy())
print('The new b2 is',b2.numpy())

Total loss = tf.Tensor([1.2213281], shape=(1,), dtype=float32)
The new w0 is [[[[ 0.7730709]]

  [[-0.0269291]]]


 [[[-0.0269291]]

  [[ 0.7730709]]]]
The new w1 is [[-0.03398073  0.36601928]
 [ 0.17884515  0.57884514]
 [ 0.27884516  0.67884517]
 [ 0.26601928  0.6660193 ]]
The new w2 is [[ 0.55835515 -0.15835512]
 [ 1.2788975  -0.6788975 ]]
The new b0 is -0.15385821
The new b1 is 0.058967665
The new b2 is 0.3
